In [1]:
import pandas as pd


In [99]:
thedate = ['date']
#dfelectricalUsage = pd.read_csv("daily_electricity_usage 2.csv",parse_dates=thedate)
#dfelectricalUsage.count()

dfElectricalUsage = pd.read_csv('daily_electricity_usage.csv',index_col=0,parse_dates=thedate)
print(dfElectricalUsage)

               date  total daily KW
Meter ID                           
1000     2009-07-14          11.203
1000     2009-07-15           8.403
1000     2009-07-16           7.225
1000     2009-07-17          11.338
1000     2009-07-18          11.306
1000     2009-07-19           2.874
1000     2009-07-20          11.600
1000     2009-07-21           4.610
1000     2009-07-22          12.436
1000     2009-07-23           4.943
1000     2009-07-24           3.675
1000     2009-07-25          11.193
1000     2009-07-26          17.749
1000     2009-07-27          11.076
1000     2009-07-28          10.525
1000     2009-07-29           9.925
1000     2009-07-30           9.935
1000     2009-07-31          17.529
1000     2009-08-01           9.045
1000     2009-08-02           2.406
1000     2009-08-03           5.740
1000     2009-08-04          12.187
1000     2009-08-05           7.627
1000     2009-08-06          10.798
1000     2009-08-07          10.823
1000     2009-08-08         

/Users/funkyapps/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [100]:
#print(dfelectricalUsage.head())  
#setting date column as index column for better lookupolloøopøløolopoplpø--
#dfelectricalUsage.set_index('date')
#print(dfelectricalUsage.tail())
#print(dfelectricalUsage.count())
#print(dfelectricalUsage.dtypes)

In [101]:
#dfelectricalUsage.drop_duplicates()   -- there are no duplicates

In [119]:
df = dfElectricalUsage.groupby('Meter ID').nunique()
print("Total # of cunsumers", df.count())

Total # of cunsumers date              6435
total daily KW    6435
dtype: int64


In [131]:
data_fix = pd.DataFrame({'Meter ID':range(1000,6435,1)})
data_fix



,Meter ID
0,1000
1,1001
2,1002
3,1003
4,1004
5,1005
6,1006
7,1007
8,1008
9,1009


In [143]:
#mean per meter ID
dfStatFeature1 = dfElectricalUsage.groupby('Meter ID')['total daily KW'].mean()
data_fix['Avg KW'] = dfStatFeature1
#print(dfStatFeature1.head())

#Total per meter ID
dfStatFeature2 = dfElectricalUsage.groupby('Meter ID')['total daily KW'].sum()
#print(dfStatFeature2.head())
data_fix['total KW'] = dfStatFeature2

#Total per meter ID
dfStatFeature3 = dfElectricalUsage.groupby('Meter ID')['total daily KW'].max()
data_fix['Max KW'] = dfStatFeature3
#print(dfStatFeature3.head())

#Total per meter ID
dfStatFeature4 = dfElectricalUsage.groupby('Meter ID')['total daily KW'].min()
data_fix['Min KW'] = dfStatFeature4
#print(dfStatFeature4.head())
print(data_fix.tail())


      Meter ID   total KW   Max KW   Min KW      Avg KW
5430      6430  12389.286   88.886    6.581   23.114340
5431      6431  15738.818  568.207  163.326  491.838062
5432      6432   6742.835   35.006    1.759   12.603430
5433      6433  19567.792   75.694    8.747   36.507075
5434      6434  12681.422   47.170    9.194   23.659369


## Adding calender features to dataset ##

In [108]:
from datetime import date
import calendar
my_date = date.today()
calendar.day_name[my_date.weekday()]

'Sunday'

In [19]:
column_1 = dfElectricalUsage.iloc[:,0]

dfElectricalUsage = pd.DataFrame({"date":column_1,
                               "year": column_1.dt.year,
                               "month": column_1.dt.month,
                               "day": column_1.dt.day,
                               "dayofyear": column_1.dt.dayofyear,
                               "weekofyear": column_1.dt.weekofyear,
                               "weekday": column_1.dt.weekday,
                               "quarter": column_1.dt.quarter,
                               "DailyUsage":dfElectricalUsage['total daily KW'] })

print(dfElectricalUsage)

               date  year  month  day  dayofyear  weekofyear  weekday  \
Meter ID                                                                
1000     2009-07-14  2009      7   14        195          29        1   
1000     2009-07-15  2009      7   15        196          29        2   
1000     2009-07-16  2009      7   16        197          29        3   
1000     2009-07-17  2009      7   17        198          29        4   
1000     2009-07-18  2009      7   18        199          29        5   
1000     2009-07-19  2009      7   19        200          29        6   
1000     2009-07-20  2009      7   20        201          30        0   
1000     2009-07-21  2009      7   21        202          30        1   
1000     2009-07-22  2009      7   22        203          30        2   
1000     2009-07-23  2009      7   23        204          30        3   
1000     2009-07-24  2009      7   24        205          30        4   
1000     2009-07-25  2009      7   25        206   

In [26]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
cal = calendar()
holidays = cal.holidays(start='2009-07-14', end='2010-10-17')
dfElectricalUsage['Holiday'] = dfElectricalUsage['date'].isin(holidays)
#print(dfelectricalUsage)
print (dfElectricalUsage[dfElectricalUsage['Holiday']== True])

               date  year  month  day  dayofyear  weekofyear  weekday  \
Meter ID                                                                
1000     2009-09-07  2009      9    7        250          37        0   
1000     2009-10-12  2009     10   12        285          42        0   
1000     2009-11-11  2009     11   11        315          46        2   
1000     2009-11-26  2009     11   26        330          48        3   
1000     2009-12-25  2009     12   25        359          52        4   
1000     2010-01-01  2010      1    1          1          53        4   
1000     2010-01-18  2010      1   18         18           3        0   
1000     2010-02-15  2010      2   15         46           7        0   
1000     2010-05-31  2010      5   31        151          22        0   
1000     2010-07-05  2010      7    5        186          27        0   
1000     2010-09-06  2010      9    6        249          36        0   
1000     2010-10-11  2010     10   11        284   